<a href="https://colab.research.google.com/github/mansi1189/Natural-Language-Processing/blob/master/ATIS_NER_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from zipfile import ZipFile
import os
import pickle
import torch
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertForTokenClassification, BertAdam
from tqdm import tqdm, trange
from seqeval.metrics import f1_score


In [2]:
!pip3 install pytorch_pretrained_bert
!pip install seqeval

     |████████████████████████████████| 133kB 4.9MB/s 


In [0]:
with ZipFile("atis.zip", 'r') as zipObj:
   zipObj.extractall()


In [0]:
def load_ds(fname='atis.train.pkl'):
    with open(fname, 'rb') as stream:
        ds,dicts = pickle.load(stream)
    print('Done  loading: ', fname)
    print('      samples: {:4d}'.format(len(ds['query'])))
    print('   vocab_size: {:4d}'.format(len(dicts['token_ids'])))
    print('   slot count: {:4d}'.format(len(dicts['slot_ids'])))
    print(' intent count: {:4d}'.format(len(dicts['intent_ids'])))
    return ds,dicts


In [0]:
# my_dict_final = []  # Create an empty dictionary
# with open('atis.train.pkl', 'rb') as f:
#     my_dict_final.append(pickle.load(f))   # Update contents of file1 to the dictionary
# with open('atis.test.pkl', 'rb') as f:
#     my_dict_final.append(pickle.load(f))   # Update contents of file2 to the dictionary
# print(my_dict_final)

In [6]:
train_ds, dicts = load_ds('atis.train.pkl')
test_ds, dicts  = load_ds('atis.test.pkl')

Done  loading:  atis.train.pkl
      samples: 4978
   vocab_size:  943
   slot count:  129
 intent count:   26
Done  loading:  atis.test.pkl
      samples:  893
   vocab_size:  943
   slot count:  129
 intent count:   26


In [0]:
#Python map() function is used to apply a function on all the elements of specified iterable and return map object. Python map object is an iterator, so we can iterate over its elements.

t2i, s2i, in2i = map(dicts.get, ['token_ids', 'slot_ids','intent_ids'])
i2t, i2s, i2in = map(lambda d: {d[k]:k for k in d.keys()}, [t2i,s2i,in2i])
#The method get() returns a value for the given key. If key is not available then returns default value None.
query, slots, intent =  map(train_ds.get, ['query', 'slot_labels', 'intent_labels'])

sentence=[]

labels=[]
z=[]

for i in range(len(query)):
  y='{}'.format(' '.join(map(i2t.get, query[i])))
  sentence.append(y)
  lab=[]
  for j in range(len(query[i])):
    z='{}'.format(i2s[slots[i][j]])
#     print(z)  
    lab.append(z)
#   print(lab) 
  labels.append(lab)

# for i in labels:
  
#   print(i)  
  

In [0]:
l=[]
for i in range(len(query)):
  
  for j in range(len(query[i])):
    z='{}'.format(i2s[slots[i][j]])
#     print(z)  
    l.append(z)
#   

In [11]:
tags_vals = list(set(s2i.keys()))
print(tags_vals)

['B-depart_date.date_relative', 'B-return_date.day_name', 'B-flight_mod', 'B-fromloc.city_name', 'B-day_number', 'B-fare_basis_code', 'I-depart_time.time_relative', 'B-flight_days', 'B-stoploc.airport_code', 'B-flight_stop', 'B-return_time.period_of_day', 'B-arrive_time.period_mod', 'B-period_of_day', 'I-depart_time.end_time', 'I-return_date.day_number', 'O', 'B-state_code', 'I-meal_code', 'I-depart_date.today_relative', 'B-restriction_code', 'I-depart_time.period_of_day', 'I-flight_number', 'B-depart_date.day_number', 'I-toloc.state_name', 'B-meal', 'I-toloc.airport_name', 'I-fare_basis_code', 'I-airline_name', 'I-round_trip', 'B-meal_code', 'B-meal_description', 'B-booking_class', 'B-toloc.state_code', 'B-or', 'I-toloc.city_name', 'B-fromloc.state_code', 'B-time_relative', 'I-fromloc.state_name', 'B-toloc.state_name', 'B-return_date.today_relative', 'B-day_name', 'B-depart_time.period_mod', 'B-airport_code', 'B-depart_time.time', 'B-time', 'B-flight_number', 'I-return_date.today_rela

In [12]:
labels[:5]

[['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-fromloc.city_name',
  'O',
  'B-depart_time.time',
  'I-depart_time.time',
  'O',
  'O',
  'O',
  'B-toloc.city_name',
  'O',
  'B-arrive_time.time',
  'O',
  'O',
  'B-arrive_time.period_of_day',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-fromloc.city_name',
  'O',
  'B-toloc.city_name',
  'O',
  'B-depart_date.day_name',
  'B-depart_time.period_of_day',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'B-flight_time',
  'I-flight_time',
  'O',
  'B-fromloc.city_name',
  'I-fromloc.city_name',
  'O',
  'O',
  'B-depart_time.time',
  'I-depart_time.time',
  'O',
  'O',
  'B-fromloc.city_name',
  'O'],
 ['O',
  'B-cost_relative',
  'O',
  'O',
  'B-fromloc.city_name',
  'O',
  'B-toloc.city_name',
  'O'],
 ['O',
  'B-round_trip',
  'I-round_trip',
  'O',
  'O',
  'B-fromloc.city_name',
  'O',
  'B-toloc.city_name',
  'B-cost_relative',
  'B-fare_amount',
  'I-fare_amount',
  'O']]

In [13]:
# Visualization

for i in range(5):
    print('{:4d}:{:>15}: {}'.format(i, i2in[intent[i][0]],
                                    ' '.join(map(i2t.get, query[i]))))
    for j in range(len(query[i])):
        print('{:>33} {:>40}'.format(i2t[query[i][j]],
                                     i2s[slots[i][j]]  ))
    print('*'*74)

   0:         flight: BOS i want to fly from boston at 838 am and arrive in denver at 1110 in the morning EOS
                              BOS                                        O
                                i                                        O
                             want                                        O
                               to                                        O
                              fly                                        O
                             from                                        O
                           boston                      B-fromloc.city_name
                               at                                        O
                              838                       B-depart_time.time
                               am                       I-depart_time.time
                              and                                        O
                           arrive                                

In [0]:
MAX_LEN = 75
bs = 32

In [0]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()


In [16]:
torch.cuda.get_device_name(0) 

'Tesla T4'

In [36]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
model.eval()

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1)
            )
          )
          (intermediate):

In [52]:
model.train()

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1)
            )
          )
          (intermediate):

In [38]:
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentence]
print(tokenized_texts[0:10])
len(tokenized_texts)

[['bo', '##s', 'i', 'want', 'to', 'fly', 'from', 'boston', 'at', '83', '##8', 'am', 'and', 'arrive', 'in', 'denver', 'at', '111', '##0', 'in', 'the', 'morning', 'e', '##os'], ['bo', '##s', 'what', 'flights', 'are', 'available', 'from', 'pittsburgh', 'to', 'baltimore', 'on', 'thursday', 'morning', 'e', '##os'], ['bo', '##s', 'what', 'is', 'the', 'arrival', 'time', 'in', 'san', 'francisco', 'for', 'the', '75', '##5', 'am', 'flight', 'leaving', 'washington', 'e', '##os'], ['bo', '##s', 'cheap', '##est', 'air', '##fare', 'from', 'tacoma', 'to', 'orlando', 'e', '##os'], ['bo', '##s', 'round', 'trip', 'fares', 'from', 'pittsburgh', 'to', 'philadelphia', 'under', '1000', 'dollars', 'e', '##os'], ['bo', '##s', 'i', 'need', 'a', 'flight', 'tomorrow', 'from', 'columbus', 'to', 'minneapolis', 'e', '##os'], ['bo', '##s', 'what', 'kind', 'of', 'aircraft', 'is', 'used', 'on', 'a', 'flight', 'from', 'cleveland', 'to', 'dallas', 'e', '##os'], ['bo', '##s', 'show', 'me', 'the', 'flights', 'from', 'pitt

4978

In [0]:
#pad_sequences is used to ensure that all sequences in a list have the same length. By default this is done by padding 0 in the beginning of each sequence until each sequence has the same length as the longest sequence.
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", padding="post")

In [39]:
input_ids[:1]

array([[ 8945,  2015,  1045,  2215,  2000,  4875,  2013,  3731,  2012,
         6640,  2620,  2572,  1998,  7180,  1999,  7573,  2012, 11118,
         2692,  1999,  1996,  2851,  1041,  2891,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0]])

In [21]:

sentence[0]

'BOS i want to fly from boston at 838 am and arrive in denver at 1110 in the morning EOS'

In [22]:
print(labels[0])

['O', 'O', 'O', 'O', 'O', 'O', 'B-fromloc.city_name', 'O', 'B-depart_time.time', 'I-depart_time.time', 'O', 'O', 'O', 'B-toloc.city_name', 'O', 'B-arrive_time.time', 'O', 'O', 'B-arrive_time.period_of_day', 'O']


In [0]:
tags = pad_sequences([[s2i.get(l) for l in lab] for lab in labels],
                     maxlen=MAX_LEN, value=s2i["O"], padding="post",dtype="long", truncating="post")

In [24]:
print(len(sentence))
print(len(labels))
print(len(tags))
print(len(input_ids))

4978
4978
4978
4978


In [0]:
attention_masks = [[float(i>0) for i in ii] for ii in input_ids]

In [0]:
tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids, tags, 
                                                            random_state=2018, test_size=0.1)
tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018, test_size=0.1)

In [0]:
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)

In [0]:
#TensorDataset:Dataset wrapping tensors.

#Each sample will be retrieved by indexing tensors along the first dimension.

train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)


#Data loader. Combines a dataset and a sampler, and provides single- or multi-process iterators over the dataset.
# sampler (Sampler, optional) – defines the strategy to draw samples from the dataset. 
# bs=32
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=bs)

In [48]:
type(train_data)

torch.utils.data.dataset.TensorDataset

In [29]:
model = BertForTokenClassification.from_pretrained("bert-base-uncased", num_labels=len(s2i))

100%|██████████| 407873900/407873900 [00:10<00:00, 37339836.61B/s]


In [49]:
model.eval()

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1)
            )
          )
          (intermediate):

In [0]:
model.cuda();

In [0]:

FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters()) 
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
optimizer = Adam(optimizer_grouped_parameters, lr=3e-5)


In [0]:


def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=2).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [33]:
epochs = 5
max_grad_norm = 1.0

for _ in trange(epochs, desc="Epoch"):
    # TRAIN loop
    model.train()
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        
        # forward pass
        loss = model(b_input_ids, token_type_ids=None,
                     attention_mask=b_input_mask, labels=b_labels)
        
        # backward pass
        loss.backward()
        
        # track train loss
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        # update parameters
        optimizer.step()
        model.zero_grad()
    # print train loss per epoch
    print("Train loss: {}".format(tr_loss/nb_tr_steps))
    # VALIDATION on validation set
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    predictions , true_labels = [], []
    for batch in valid_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        
        with torch.no_grad():
            tmp_eval_loss = model(b_input_ids, token_type_ids=None,
                                  attention_mask=b_input_mask, labels=b_labels)
            logits = model(b_input_ids, token_type_ids=None,
                           attention_mask=b_input_mask)
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
        true_labels.append(label_ids)
        
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        eval_loss += tmp_eval_loss.mean().item()
        eval_accuracy += tmp_eval_accuracy
        
        nb_eval_examples += b_input_ids.size(0)
        nb_eval_steps += 1
    eval_loss = eval_loss/nb_eval_steps
    print("Validation loss: {}".format(eval_loss))
    print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
    pred_tags = [tags_vals[p_i] for p in predictions for p_i in p]
    valid_tags = [tags_vals[l_ii] for l in true_labels for l_i in l for l_ii in l_i]
    print("F1-Score: {}".format(f1_score(pred_tags, valid_tags)))

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Train loss: 0.9632604002952576


Epoch:  20%|██        | 1/5 [01:12<04:48, 72.13s/it]

Validation loss: 0.4350757710635662
Validation Accuracy: 0.9296990740740741
F1-Score: 0.931899448850652
Train loss: 0.33794698651347843


Epoch:  40%|████      | 2/5 [02:30<03:42, 74.11s/it]

...

Validation loss: 0.14590048557147384
Validation Accuracy: 0.8647366898148148
F1-Score: 0.8689079027150846
Train loss: 0.12241391997252192


Epoch:  80%|████████  | 4/5 [05:07<01:16, 76.28s/it]

Validation loss: 0.11382047645747662
Validation Accuracy: 0.8604687499999998
F1-Score: 0.8657583082386939
Train loss: 0.08624604433508856


Epoch: 100%|██████████| 5/5 [06:25<00:00, 76.85s/it]

Validation loss: 0.09468488593120128
Validation Accuracy: 0.8607002314814814
F1-Score: 0.8666306403674683


In [0]:
model.eval()
predictions = []
true_labels = []
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
for batch in valid_dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    with torch.no_grad():
        tmp_eval_loss = model(b_input_ids, token_type_ids=None,
                              attention_mask=b_input_mask, labels=b_labels)
        logits = model(b_input_ids, token_type_ids=None,
                       attention_mask=b_input_mask)
        
    logits = logits.detach().cpu().numpy()
    predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
    label_ids = b_labels.to('cpu').numpy()
    true_labels.append(label_ids)
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)

    eval_loss += tmp_eval_loss.mean().item()
    eval_accuracy += tmp_eval_accuracy

    nb_eval_examples += b_input_ids.size(0)
    nb_eval_steps += 1

pred_tags = [[tags_vals[p_i] for p_i in p] for p in predictions]
valid_tags = [[tags_vals[l_ii] for l_ii in l_i] for l in true_labels for l_i in l ]
print("Validation loss: {}".format(eval_loss/nb_eval_steps))
print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
print("Validation F1-Score: {}".format(f1_score(pred_tags, valid_tags)))

Validation loss: 0.024513516516890377
Validation Accuracy: 0.9942129629629628
Validation F1-Score: 0.9383791858583344
